In [4]:
import pandas as pd
import flair
from bpemb import BPEmb
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cdist
from sklearn.decomposition import TruncatedSVD
import regex as re
#import spacy

In [11]:
full_files_assignment = [f"data/assignment/{x}/assignment.txt" for x in os.listdir('data/assignment')]
full_files_data_assignment = [open(x).read() for x in full_files_assignment if not 'annot' in x and not '.DS' in x]
full_files = [f"data/resumes/{x}" for x in os.listdir('data/resumes')]
full_files_data = [open(x).read() for x in full_files]

all_text = ' '.join(full_files_data_assignment + full_files_data)

In [19]:
all_text[:200]

'AD System designer / System Architect - Automotive, Referens 20922\nAnsök här\nUppdragsbeskrivning och krav\nWe are now looking for a System designer/System Architect to our Clients autonomous drive and '

In [98]:
multi_space = re.compile("[_ \t]+")
non_ascii = re.compile("[^\x00-\x7FåäöÅÄÖ\s\n\p{Punct}]+")
punct = re.compile('([\p{Punct}\n])+')
#splitter = re.compile("[\n\.]")
#t = all_text.replace('/', ' ')
t = non_ascii.sub('', all_text)
t = punct.sub(' \g<1> ', t)
t = multi_space.sub(' ', t)
t = t.replace('\n', '.')
t = t.lower()

In [99]:
t[:300]

'ad system designer / system architect - automotive , referens 20922 . ansök här . uppdragsbeskrivning och krav . we are now looking for a system designer / system architect to our clients autonomous drive and active safety function department . you have documented experience of system design within '

In [49]:
sentences = [x for x in splitter.split(t) if len(x) > 3]

In [51]:
sentences_subword = [bpemb.encode(x) for x in sentences]

In [37]:
bpemb = BPEmb(lang='en')

downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.model
100%|██████████| 400869/400869 [00:00<00:00, 2583033.81B/s]
downloading https://nlp.h-its.org/bpemb/en/en.wiki.bpe.vs10000.d100.w2v.bin.tar.gz
100%|██████████| 3784656/3784656 [00:00<00:00, 11670195.12B/s]


In [38]:
bpemb

BPEmb(lang=en, vs=10000, dim=100)

In [41]:
bpemb.emb

In [42]:
from gensim.models import Word2Vec

In [56]:
model = Word2Vec(sentences=sentences_subword, window=5, min_count=1, workers=4)

In [59]:
model.train(sentences_subword, total_examples=len(sentences_subword),epochs=1)

(791534, 912786)

In [63]:
model.save("word2vec.wordvectors")

In [74]:
model.wv.most_similar('comp', topn=10) 

[('oons', 0.9659119248390198),
 ('▁comics', 0.9635868072509766),
 ('agan', 0.9604225158691406),
 ('▁clinical', 0.9601695537567139),
 ('amo', 0.9595756530761719),
 ('▁moscow', 0.9584916830062866),
 ('▁syria', 0.957403302192688),
 ('ounced', 0.9567681550979614),
 ('▁dave', 0.9562561511993408),
 ('▁disk', 0.9558833837509155)]

In [ ]:
x

In [73]:
bpeCust = BPEmb(lang='en', emb_file='word2vec.wordvectors')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte